In [1]:
import folium
import pandas as pd
from string import Template

In [2]:
df = pd.read_csv("log_sheet.csv").fillna("").astype(str)

In [3]:
df = df.rename(columns = {"Name*": "Name"})

folk = ["Scott", "Matthew", "Amy"]

for col in ["Metres"] + folk:
    df[col] = df[col].apply(lambda x: x.split(".")[0])

KeyError: 'Amy'

In [ ]:
table_template = Template("""<!DOCTYPE html>
                            <html lang="en">
                            
                            
                            <style>
                              table {
                                    width: 100%;
                                    border-collapse: collapse;
                                    font-family: Arial, sans-serif;
                                    box-shadow: 0 2px 8px rgba(0,0,0,0.1);
                                    background: #fff;
                                    table-layout: fixed;
                                  }

                                    th {
                                        border: 1px solid #ddd;
                                        background-color: #f7f7f7;
                                        padding: 12px 15px;
                                        text-align: left;
                                        font-weight: 600;
                                      }
                            
                                td {
                                    border: 1px solid #ddd;
                                    padding: 10px 15px;
                                    text-align: left;
                                  }

                            tbody tr:hover {
                                            background-color: #f1f1f1;
                                          }
                            </style>

                            <script>
                            function filterByColumnCellIds() {
                              const filter1 = document.getElementById("filterCol1").value;
                              const filter2 = document.getElementById("filterCol2").value;
                            
                              const rows = document.querySelectorAll("#mountainTable tbody tr");
                            
                              rows.forEach(row => {
                                const cell1 = row.cells[2]; 
                                const cell2 = row.cells[3]; 
                            
                                const matchCol1 = !filter1 || cell1.id === filter1;
                                const matchCol2 = !filter2 || cell2.id === filter2;
                            
                                row.style.display = (matchCol1 && matchCol2) ? "" : "none";
                              });
                            }   
                            </script>

                                  <script>
                          document.addEventListener("DOMContentLoaded", function () {
                            const searchInput = document.getElementById('searchInput');
                            const rows = document.querySelectorAll('#mountainTable tbody tr');
                        
                            searchInput.addEventListener('input', function () {
                              const filter = searchInput.value.toLowerCase();
                              rows.forEach(row => {
                                const nameText = row.cells[0].textContent.toLowerCase();
                                row.style.display = nameText.includes(filter) ? '' : 'none';
                              });
                            });
                          });
                        </script>

                        <head>
                         <base href="https://raw.githubusercontent.com/mprallison/munro_bagger/main/">
                        </head>
                    

                        <body>

                        <div class="table-content" id="tableContent">
                        <table id="mountainTable">
                        <colgroup>
                        <col style="width: 100px">
                        <col style="width: 80px">
                        <col style="width: 160px">
                        <col style="width: 160px">
                        </colgroup>
                            <thead>
                                <tr>
                                    <th>Name (282)<br> <input type="text" id="searchInput" placeholder="🔍" style="width: 100px"></th>
                                    
                                    <th>Height</th>
                                    <th>Matthew $matthew_total
                                    <select id="filterCol1" onchange="filterByColumnCellIds()">
                                      <option value="">All</option>
                                      <option value="bag">Bags</option>
                                      <option value="gap">Gaps</option>
                                    </select>
                                    </th>

                                    <th>Scott $scott_total
                                    <select id="filterCol2" onchange="filterByColumnCellIds()">
                                      <option value="">All</option>
                                      <option value="bag">Bags</option>
                                      <option value="gap">Gaps</option>
                                    </select>
                                    </th>
                                </tr>
                            </thead>
                            <tbody>
                                $rows
                            </tbody>
                        </table>
                         </div>
                        </body>
                        </html>
                        """)

In [ ]:
row_template = Template(
                        """<tr>
                            <td>$name</td>
                            <td>$height</td>
                            <td id = "$matthew_status"> $matthew</td>
                            <td id = "$scott_status"> $scott</td>
                          </tr>
                          """)

In [ ]:
rows_html = ""

for _, row in df.iterrows():

    if row["Matthew"] != "":
        matthew = '<img src="images/matthew.png" width="64" height="80">'
        matthew_status = "bag"
    else:
        matthew = '<img src="images/peak.png" width="80" height="80">'
        matthew_status = "gap"
        
    if row["Scott"] != "":
        scott = '<img src="images/scott.png" width="64" height="80">'
        scott_status = "bag"
    else:
        scott = '<img src="images/peak.png" width="80" height="80">'
        scott_status = "gap"

    if row["Amy"] != "":
        scott = '<img src="images/scott.png" width="64" height="80">'
        scott_status = "bag"
    else:
        scott = '<img src="images/peak.png" width="80" height="80">'
        scott_status = "gap"

    row = row_template.substitute({'name': row["Name"],
                            'lat': row["Latitude"],
                            'long': row["Longitude"],
                            'height': row["Metres"]+"m",
                            'matthew': matthew,
                            'matthew_status': matthew_status,
                            'scott': scott,
                            'scott_status': scott_status,
                            'amy': amy,
                            'amy_status': amy_status,
                            })


    rows_html += row

In [ ]:
table_html = table_template.substitute({"rows": rows_html,
                                        "matthew_total": f"({len(df[df["Matthew"] != ""])})",
                                         "scott_total": f"({len(df[df["Scott"] != ""])})",    
                                        "amy_total": f"({len(df[df["Amy"] != ""])})",   
                                       })

In [ ]:
with open("table.html", "w", encoding="utf-8") as file:
    file.write(table_html)